In [1]:
pip install plotly


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import plotly.express as px
import pandas as pd

# Specify the path to your transcript file
file_path = "/Users/norika_machome/Library/CloudStorage/OneDrive-UniversiteitLeiden/Semester1/4.MS_FRI/final_project/speechProcess/audio8.txt"

# Read the transcript file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Create a list to store the timestamp, speaker, text, and word count information
transcript = []

# Get a list of all unique speakers
all_speakers = set()

# Iterate through the lines in the file and extract information
for line in lines:
    # Assuming the speaker information is in a specific format, adapt as needed
    if line.startswith("[") and "]" in line:
        speaker = line.split(" ")[1]
        all_speakers.add(speaker)

# Create initial dataframe with all speakers and zero cumulative word count
initial_data = [{'timestamp': '', 'speaker': speaker, 'cumulative_word_count': 0} for speaker in all_speakers]
df_initial = pd.DataFrame(initial_data)

# Initialize variables for tracking word count over time for each speaker
current_word_counts = {speaker: 0 for speaker in all_speakers}

# Iterate through the lines in the file and extract information
for line in lines:
    # Assuming the speaker information is in a specific format, adapt as needed
    if line.startswith("[") and "]" in line:
        timestamp = line.split("]")[0][1:]
        text = " ".join(line.split(" ")[2:]).strip()

        # Calculate word count
        word_count = len(text.split())

        # Update word count for each speaker
        for speaker in all_speakers:
            # Check if the current speaker matches the loop speaker
            if speaker in line:
                # Update cumulative word count for the current speaker
                current_word_counts[speaker] += word_count

                # Append information to the transcript list
                transcript.append({
                    "timestamp": timestamp,
                    "speaker": speaker,
                    "text": text,
                    "word_count": word_count,
                    "cumulative_word_count": current_word_counts[speaker]
                })
            else:
                # If the speaker doesn't match, use zero word count
                transcript.append({
                    "timestamp": timestamp,
                    "speaker": speaker,
                    "text": "",
                    "word_count": 0,
                    "cumulative_word_count": current_word_counts[speaker]
                })

# Create a DataFrame from the transcript data
df = pd.DataFrame(transcript)

# Convert timestamp to datetime for better plotting
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%M:%S.%f')

# Convert the timestamp to string for animation
df['timestamp_str'] = df['timestamp'].dt.strftime('%M:%S.%f')

#Define the mapping of original speaker names to new names
speaker_mapping = {'SPEAKER_00:': 'no', 'SPEAKER_01:': 'ri', 'SPEAKER_02:': 'ka'}

# Replace speaker names using the map function
df['speaker'] = df['speaker'].map(speaker_mapping)
df

,timestamp,speaker,text,word_count,cumulative_word_count,timestamp_str
0,1900-01-01 00:00:00.220,no,,0,0,00:00.220000
1,1900-01-01 00:00:00.220,ri,,0,0,00:00.220000
2,1900-01-01 00:00:00.220,ka,"Hi, this is Bill Wale in the Host of Good Fell...",11,11,00:00.220000
3,1900-01-01 00:00:02.440,no,,0,0,00:02.440000
4,1900-01-01 00:00:02.440,ri,,0,0,00:02.440000
...,...,...,...,...,...,...
235,1900-01-01 00:02:53.940,ri,,0,38,02:53.940000
236,1900-01-01 00:02:53.940,ka,Is this a set of circumstances unique,7,499,02:53.940000
237,1900-01-01 00:02:56.670,no,,0,51,02:56.670000
238,1900-01-01 00:02:56.670,ri,,0,38,02:56.670000


In [53]:
# Group by speaker and timestamp and concatenate the text
full_text_df = df.groupby(['speaker', 'timestamp'])['text'].apply(lambda x: ' '.join(x)).reset_index()

# Concatenate all text from different speakers and timestamps
full_text = ' '.join(full_text_df['text'])

# Print or use the full_text as needed
print(full_text)

         It's a witch hunt, it's just a continuation of a witch hunt. They want to silence you, they want to silence you, and they mean silence, because I have four of them now. I feel like I mean, this is not even possible for. Over the next less couple of months.                                                                                                             This is such a trade bill. Thank you so much. I'm a huge fan. And there are some episodes I can only listen to once, but I get to at least one. So it's great to be here.                                 Hi, this is Bill Wale in the Host of Good Fellows. Thanks for listening to the audio version of the show, but we want to let you know that Good Fellows is primarily a video production, and you're missing a lot of extra features by only listening to our show. Give it a look by going to Hoover.org forward slash Good Fellows to see what you're missing. Thanks.      It's Tuesday, August 22nd, 2023, and welcome back to Good F

In [54]:
# Create an animated bar plot with Plotly Express
fig = px.bar(df, x='speaker', y='cumulative_word_count', color='speaker',
             animation_frame='timestamp_str',
             title='Cumulative Word Count Over Time for Each Speaker',
             labels={'timestamp_str': 'Timestamp', 'cumulative_word_count': 'Cumulative Word Count'},
             category_orders={'speaker': sorted(df['speaker'].unique())},
             )

# Update layout for better readability
fig.update_layout(xaxis=dict(title='Speaker'),
                  yaxis=dict(title='Cumulative Word Count', range=[0, 500]),
                  )

# Show the plot
fig.show()

In [55]:
pip install sumy


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 485.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.0 MB/s eta 0:00:0000:0100:01m
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 5.6 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21713 sha256=ea570701c08f37e8b123d2dce39d043ff9a37c620982f62daf76164c64dc6552
  Stored in directory: /Users/norika_machome/Library/Caches/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
Successfully built breadability

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Libra

In [57]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk
nltk.download('punkt')

# Use the full_text variable for summarization
text_to_summarize = full_text

# Choose the language and create a parser
language = "english"
parser = PlaintextParser.from_string(text_to_summarize, Tokenizer(language))

# Create an LSA (Latent Semantic Analysis) summarizer
lsa_summarizer = LsaSummarizer()

# Summarize the text
summary = lsa_summarizer(parser.document, sentences_count=3)  # You can adjust the number of sentences in the summary

# Print the summarized text
for sentence in summary:
    print(sentence)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/norika_machome/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


And of course, our resident eternal optimist, Lieutenant General HR, I'm McMaster.
If one of these trials with Donald Trump begins next March, let's say, that is peak primary system.
Good luck getting attention if the cameras are all falling Donald Trump into a courtroom.


In [62]:
from gensim import corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK stopwords
import nltk
nltk.download('stopwords')

# Preprocess the text
stop_words = set(stopwords.words('english'))
tokens = word_tokenize(full_text.lower())
filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary([filtered_tokens])

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(filtered_tokens)]

# Train the LDA model
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

# Print the topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)


(0, '0.005*"andy" + 0.005*"good" + 0.005*"fellows" + 0.005*"want" + 0.005*"trump"')
(1, '0.005*"andy" + 0.005*"fellows" + 0.005*"good" + 0.005*"trump" + 0.005*"want"')
(2, '0.021*"good" + 0.021*"fellows" + 0.021*"andy" + 0.015*"want" + 0.015*"trump"')
(3, '0.005*"andy" + 0.005*"fellows" + 0.005*"good" + 0.005*"want" + 0.005*"trump"')
(4, '0.005*"fellows" + 0.005*"andy" + 0.005*"trump" + 0.005*"good" + 0.005*"want"')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/norika_machome/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
